In [1]:
import json

In [ ]:
def read_dataset(filepath):
    

In [36]:
filepath = 'output/prof_gender.json'
distributions = json.load(open(filepath, 'r'))['distributions']

In [9]:
import re
import numpy as np
import pandas as pd
from pprint import pprint

# Gensim
import gensim
import gensim.corpora as corpora
from gensim.utils import simple_preprocess
from gensim.models import CoherenceModel

# spacy for lemmatization
import spacy

# Plotting tools
import pyLDAvis
import pyLDAvis.gensim_models  # don't skip this
import matplotlib.pyplot as plt
%matplotlib inline

# Enable logging for gensim - optional
import logging
logging.basicConfig(format='%(asctime)s : %(levelname)s : %(message)s', level=logging.ERROR)

import warnings
warnings.filterwarnings("ignore",category=DeprecationWarning)

/Users/peterzhang/opt/anaconda3/lib/python3.8/site-packages/ipykernel/ipkernel.py:287: DeprecationWarning: `should_run_async` will not call `transform_cell` automatically in the future. Please pass the result to `transformed_cell` argument and any exception that happen during thetransform in `preprocessing_exc_tuple` in IPython 7.17 and above.
  and should_run_async(code)


In [11]:
import nltk
nltk.download('stopwords')

from nltk.corpus import stopwords
stop_words = stopwords.words('english')
stop_words.extend(['from', 'subject', 're', 'edu', 'use'])

[nltk_data] Downloading package stopwords to
[nltk_data]     /Users/peterzhang/nltk_data...
[nltk_data]   Unzipping corpora/stopwords.zip.


In [37]:
distributions.keys()

dict_keys(['woman', 'man'])

In [50]:
data = distributions['woman']

In [51]:

from nltk.corpus import stopwords
stop_words = stopwords.words('english')
stop_words.extend(['from', 'subject', 're', 'edu', 'use'])

In [52]:
def sent_to_words(sentences):
    for sentence in sentences:
        yield(gensim.utils.simple_preprocess(str(sentence), deacc=True))  # deacc=True removes punctuations

data_words = list(sent_to_words(data))

In [53]:
# Build the bigram and trigram models
bigram = gensim.models.Phrases(data_words, min_count=5, threshold=100) # higher threshold fewer phrases.
trigram = gensim.models.Phrases(bigram[data_words], threshold=100)  

# Faster way to get a sentence clubbed as a trigram/bigram
bigram_mod = gensim.models.phrases.Phraser(bigram)
trigram_mod = gensim.models.phrases.Phraser(trigram)

# See trigram example
print(trigram_mod[bigram_mod[data_words[0]]])

['jans', 'is', 'great', 'she', 'is', 'fantastic', 'teacher', 'and', 'her', 'class', 'was', 'both', 'interesting', 'and', 'lively', 'she', 'does', 'expect', 'you', 'to', 'read', 'lot', 'out', 'of', 'the', 'book', 'and', 'other', 'sources', 'as', 'well', 'she', 'is', 'very', 'lively', 'teacher', 'and', 'you', 'can', 'tell', 'she', 'loves', 'the', 'subject', 'she', 'teaches']


In [54]:
# Define functions for stopwords, bigrams, trigrams and lemmatization
def remove_stopwords(texts):
    return [[word for word in simple_preprocess(str(doc)) if word not in stop_words] for doc in texts]

def make_bigrams(texts):
    return [bigram_mod[doc] for doc in texts]

def make_trigrams(texts):
    return [trigram_mod[bigram_mod[doc]] for doc in texts]

def lemmatization(texts, allowed_postags=['NOUN', 'ADJ', 'VERB', 'ADV']):
    """https://spacy.io/api/annotation"""
    texts_out = []
    for sent in texts:
        doc = nlp(" ".join(sent)) 
        texts_out.append([token.lemma_ for token in doc if token.pos_ in allowed_postags])
    return texts_out

In [55]:
!python -m spacy download en_core_web_sm

     |████████████████████████████████| 13.9 MB 7.1 MB/s eta 0:00:01
✔ Download and installation successful
You can now load the package via spacy.load('en_core_web_sm')


In [56]:
# Remove Stop Words
data_words_nostops = remove_stopwords(data_words)

# Form Bigrams
data_words_bigrams = make_bigrams(data_words_nostops)

# Initialize spacy 'en' model, keeping only tagger component (for efficiency)
# python3 -m spacy download en
nlp = spacy.load("en_core_web_sm", disable=['parser', 'ner'])

# Do lemmatization keeping only noun, adj, vb, adv
data_lemmatized = lemmatization(data_words_bigrams, allowed_postags=['NOUN', 'ADJ', 'VERB', 'ADV'])

print(data_lemmatized[:1])

[['great', 'fantastic', 'teacher', 'class', 'interesting', 'lively', 'expect', 'read', 'lot', 'book', 'source', 'well', 'lively', 'teacher', 'tell', 'love', 'teach']]


In [57]:
# Create Dictionary
id2word = corpora.Dictionary(data_lemmatized)

# Create Corpus
texts = data_lemmatized

# Term Document Frequency
corpus = [id2word.doc2bow(text) for text in texts]

# View
print(corpus[:1])


[[(0, 1), (1, 1), (2, 1), (3, 1), (4, 1), (5, 1), (6, 2), (7, 1), (8, 1), (9, 1), (10, 1), (11, 1), (12, 2), (13, 1), (14, 1)]]


In [58]:
# Build LDA model
lda_model = gensim.models.ldamodel.LdaModel(corpus=corpus,
                                           id2word=id2word,
                                           num_topics=5, 
                                           random_state=100,
                                           update_every=1,
                                           chunksize=100,
                                           passes=10,
                                           alpha='auto',
                                           per_word_topics=True)

In [59]:
# Print the Keyword in the 10 topics
pprint(lda_model.print_topics())
doc_lda = lda_model[corpus]

[(0,
  '0.042*"ever" + 0.038*"way" + 0.034*"think" + 0.034*"bad" + 0.029*"math" + '
  '0.018*"people" + 0.015*"day" + 0.015*"seem" + 0.014*"life" + 0.013*"rude"'),
 (1,
  '0.055*"professor" + 0.054*"great" + 0.053*"teacher" + 0.052*"student" + '
  '0.039*"help" + 0.034*"learn" + 0.028*"love" + 0.025*"always" + 0.021*"fun" '
  '+ 0.021*"want"'),
 (2,
  '0.032*"look" + 0.026*"art" + 0.025*"fine" + 0.019*"college" + '
  '0.017*"probably" + 0.017*"school" + 0.016*"spanish" + 0.015*"chapter" + '
  '0.014*"suffer" + 0.014*"communicate"'),
 (3,
  '0.040*"lady" + 0.032*"ready" + 0.022*"hate" + 0.022*"issue" + 0.018*"other" '
  '+ 0.017*"spend" + 0.017*"catch" + 0.015*"waste" + 0.015*"back" + '
  '0.014*"writing"'),
 (4,
  '0.086*"class" + 0.036*"take" + 0.023*"make" + 0.021*"test" + 0.018*"good" + '
  '0.018*"get" + 0.018*"easy" + 0.016*"work" + 0.015*"go" + 0.015*"really"')]


In [60]:
# Compute Perplexity
print('\nPerplexity: ', lda_model.log_perplexity(corpus))  # a measure of how good the model is. lower the better.

# Compute Coherence Score
coherence_model_lda = CoherenceModel(model=lda_model, texts=data_lemmatized, dictionary=id2word, coherence='c_v')
coherence_lda = coherence_model_lda.get_coherence()
print('\nCoherence Score: ', coherence_lda)


Perplexity:  -6.758948353834929

Coherence Score:  0.35368732250932094


In [61]:
# Visualize the topics
pyLDAvis.enable_notebook()
vis = pyLDAvis.gensim_models.prepare(lda_model, corpus, id2word)
vis

PreparedData(topic_coordinates=              x         y  topics  cluster       Freq
topic                                                
4      0.357133 -0.184677       1        1  61.431291
1      0.122830  0.331486       2        1  18.468551
0     -0.173614 -0.134502       3        1  11.017355
2     -0.152978 -0.003638       4        1   4.592845
3     -0.153372 -0.008669       5        1   4.489958, topic_info=              Term         Freq        Total Category  logprob  loglift
1            class  5449.000000  5449.000000  Default  30.0000  30.0000
51       professor  1038.000000  1038.000000  Default  29.0000  29.0000
4            great  1032.000000  1032.000000  Default  28.0000  28.0000
12         teacher  1416.000000  1416.000000  Default  27.0000  27.0000
55         student   981.000000   981.000000  Default  26.0000  26.0000
...            ...          ...          ...      ...      ...      ...
49     personality    27.087629    27.827914   Topic5  -5.1378   3.0764
50    professional    23.916054    24.657809   Topic5  -5.2623   3.0728
1005     obviously    23.545991    24.290257   Topic5  -5.2779   3.0722
769    requirement    22.485502    23.234387   Topic5  -5.3240   3.0706
179    power_point    22.556630    23.345995   Topic5  -5.3208   3.0689

[203 rows x 6 columns], token_table=      Topic      Freq        Term
term                             
245       3  0.997101        able
1346      4  0.977823  accessible
926       4  0.977481         act
56        2  0.996979    actually
71        1  0.998309        also
...     ...       ...         ...
164       2  0.997344       woman
314       2  0.995440   wonderful
38        1  0.999113        work
296       5  0.982282     writing
421       4  0.992734        year

[191 rows x 3 columns], R=30, lambda_step=0.01, plot_opts={'xlab': 'PC1', 'ylab': 'PC2'}, topic_order=[5, 2, 1, 3, 4])